---
EDA
---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio


In [2]:
cl_data = pd.read_csv("cover_letter_input_data copy.csv")
cl_data.head()

,job_description,user_experience,gender,old_output
0,'<p>Подразделение искусственного интеллекта и ...,"""ФИО: Akimali Edige \nДолжность: Младший Pytho...",NaN,"Добрый день!\n\nМеня зовут Эдиге, и я хочу отк..."
1,'<p>В Angular-комьюнити Т‑Банка более 300 проф...,"""ФИО: Тоджиев Диёр Фаридунович\nДолжность: Fro...",NaN,"Добрый день!\n\nМеня зовут Диёр, и я решил отк..."
2,'<p><strong>About Us</strong></p><p>V4Scale is...,"""ФИО: Akimali Edige \nДолжность: Junior Python...",NaN,"Добрый день!\n\nМеня зовут Акимали, и я хочу о..."
3,'<strong>Обязанности:</strong> <ul> <li>Создан...,'ФИО: Чакветадзе Давид \nДолжность: Тестировщи...,NaN,"Добрый день!\n\nМеня зовут Давид, и я хочу отк..."
4,'<strong>Эта вакансия размещена в рамках HR-на...,"""ФИО: Тоджиев Диёр Фаридунович\nДолжность: Fro...",NaN,"Добрый день!\n\nМеня зовут Диёр, и я решил отк..."


In [3]:
# Временный мок User и app
import types
import sys

app = types.ModuleType("app")
models = types.ModuleType("app.models")
user_module = types.ModuleType("app.models.user")

class User:
    def __init__(self, user_id):
        self.user_id = user_id

user_module.User = User

# Регистрируем "пакеты" в sys.modules
sys.modules["app"] = app
sys.modules["app.models"] = models
sys.modules["app.models.user"] = user_module

In [4]:
import httpx
from sqlalchemy.ext.asyncio import AsyncSession
from app.models.user import User
from sqlalchemy import select
from fastapi import HTTPException
import re
from typing import List, Dict, Tuple


class RecommendationsNotFoundError(Exception):
    pass

class ResumePublishError(Exception):
    pass

class ResumeNotFoundError(HTTPException):
    pass

async def get_http_client():
    async with httpx.AsyncClient(timeout=120.0) as client:
        yield client


# Helper functions
async def get_user_or_404(db: AsyncSession, user_id: str) -> User:
    query = select(User).where(User.user_id == user_id)
    result = await db.execute(query)
    user = result.scalar_one_or_none()

    if not user:
        raise HTTPException(status_code=404, detail="User not found")

    return user


def detect_mixed_language_anomalies(text: str) -> Dict[str, any]:
    """
    Обнаруживает аномалии смешанного языка в тексте, когда русские и английские символы
    смешиваются в одном слове (например, 'вычисляtь', 'pochemu' вместо 'почему').

    Args:
        text (str): Текст для анализа

    Returns:
        Dict: Словарь с результатами анализа:
            - has_anomalies (bool): Есть ли аномалии
            - anomalies (List[Dict]): Список найденных аномалий
            - anomaly_count (int): Количество аномалий
            - anomaly_rate (float): Процент аномалий от общего количества слов
    """
    # Регулярные выражения для определения языков
    cyrillic_pattern = re.compile(r'[а-яё]', re.IGNORECASE)
    latin_pattern = re.compile(r'[a-z]', re.IGNORECASE)

    # Разбиваем текст на слова (учитываем различные разделители)
    words = re.findall(r'\b\w+\b', text.lower())

    anomalies = []
    total_words = len(words)

    for word in words:
        if len(word) < 2:  # Пропускаем слишком короткие слова
            continue

        # Проверяем наличие символов обоих алфавитов в одном слове
        has_cyrillic = bool(cyrillic_pattern.search(word))
        has_latin = bool(latin_pattern.search(word))

        if has_cyrillic and has_latin:
            # Дополнительная проверка: слово должно содержать минимум 2 символа каждого алфавита
            cyrillic_chars = len(cyrillic_pattern.findall(word))
            latin_chars = len(latin_pattern.findall(word))

            if cyrillic_chars >= 2 and latin_chars >= 2:
                anomalies.append({
                    'word': word,
                    'position': text.lower().find(word),
                    'cyrillic_chars': cyrillic_chars,
                    'latin_chars': latin_chars,
                    'anomaly_type': 'mixed_script'
                })

    # Дополнительная проверка на транслитерацию (например, 'pochemu' вместо 'почему')
    transliteration_anomalies = detect_transliteration_anomalies(text)
    anomalies.extend(transliteration_anomalies)

    anomaly_count = len(anomalies)
    anomaly_rate = (anomaly_count / total_words * 100) if total_words > 0 else 0

    return {
        'has_anomalies': anomaly_count > 0,
        'anomalies': anomalies,
        'anomaly_count': anomaly_count,
        'anomaly_rate': round(anomaly_rate, 2),
        'total_words': total_words
    }


def detect_transliteration_anomalies(text: str) -> List[Dict]:
    """
    Обнаруживает аномалии транслитерации, когда русские слова написаны латинскими буквами.

    Args:
        text (str): Текст для анализа

    Returns:
        List[Dict]: Список найденных аномалий транслитерации
    """
    # Словарь для проверки транслитерации (латиница -> кириллица)
    transliteration_map = {
        'pochemu': 'почему',
        'kak': 'как',
        'chto': 'что',
        'gde': 'где',
        'kogda': 'когда',
        'pochemu': 'почему',
        'kakoy': 'какой',
        'kakaya': 'какая',
        'kakoe': 'какое',
        'kakie': 'какие',
        'kto': 'кто',
        'chego': 'чего',
        'chemu': 'чему',
        'chem': 'чем',
        'kogo': 'кого',
        'komu': 'кому',
        'kom': 'ком',
        'kakogo': 'какого',
        'kakuyu': 'какую',
        'kakogo': 'какого',
        'kakih': 'каких',
        'kakogo': 'какого',
        'kakuyu': 'какую',
        'kakogo': 'какого',
        'kakih': 'каких'
    }

    anomalies = []
    words = re.findall(r'\b\w+\b', text.lower())

    for word in words:
        if word in transliteration_map:
            anomalies.append({
                'word': word,
                'correct_word': transliteration_map[word],
                'position': text.lower().find(word),
                'anomaly_type': 'transliteration'
            })

    return anomalies


def get_anomaly_severity(anomaly_rate: float) -> str:
    """
    Определяет уровень серьезности аномалий на основе их частоты.

    Args:
        anomaly_rate (float): Процент аномалий

    Returns:
        str: Уровень серьезности ('low', 'medium', 'high', 'critical')
    """
    if anomaly_rate == 0:
        return 'none'
    elif anomaly_rate < 5:
        return 'low'
    elif anomaly_rate < 15:
        return 'medium'
    elif anomaly_rate < 30:
        return 'high'
    else:
        return 'critical'

In [5]:
cl_data = cl_data.loc[:, ~cl_data.columns.duplicated()]

In [15]:
import random
import pandas as pd

IT_SKILLS = {
    "python", "java", "c", "cpp", "csharp", "go", "rust", "kotlin", "swift", "r",
    "scala", "ruby", "php", "typescript", "js", "dart", "matlab",
    "julia", "perl", "bash", "shell", "objective-c",

    "sql", "nosql", "mysql", "postgres", "sqlite", "oracle", "mongodb",
    "cassandra", "redis", "elasticsearch", "firebase", "clickhouse", "snowflake",
    "bigquery", "hive", "hbase", "pandas", "numpy", "scipy", "sklearn",
    "tensorflow", "pytorch", "keras", "xgboost", "lightgbm", "catboost",
    "statsmodels", "nltk", "spacy",

    "linux", "ubuntu", "centos", "redhat", "debian", "windows", "macos", "freebsd",
    "tcp", "ip", "dns", "dhcp", "http", "https", "ssh", "ftp", "vpn",
    "active", "directory", "ldap", "git", "github", "gitlab", "bitbucket",

    "aws", "ec2", "s3", "rds", "lambda", "cloudformation",
    "azure", "gcp", "pubsub", "dataflow", "vertex", "ai",
    "heroku", "digitalocean", "openstack",

    "docker", "kubernetes", "openshift", "helm", "terraform",
    "ansible", "puppet", "chef", "jenkins", "github", "actions",
    "gitlab", "cicd", "circleci", "travisci",

    "react", "angular", "vue", "svelte", "nextjs", "nuxtjs",
    "django", "flask", "fastapi", "spring", "express", "nestjs",
    "rails", "aspnet", "html", "css", "sass", "scss",
    "tailwind", "bootstrap", "rest", "graphql", "grpc", "websockets",

    "tableau", "powerbi", "qlikview", "looker", "superset", "metabase",
    "excel", "airflow", "luigi", "dbt",

    "hadoop", "spark", "kafka", "storm", "flink", "beam",
    "databricks", "dask", "ray",

    "pytest", "junit", "selenium", "cypress", "postman", "jmeter", "playwright",

    "kali", "wireshark", "metasploit", "burp", "owasp",
    "siem", "splunk", "elk", "firewalls", "ids", "ips",

    "unity", "unreal", "figma", "adobe", "jira", "confluence",
    "trello", "notion", "agile", "scrum", "kanban"
}


# список слов для gender mismatch
GENDER_MISMATCH_WORDS = ["готовила", "вдохновила", "готова", "готовимся", "увеличила", "увеличил", "уменьшила", "уменьшил","сократила", "сократил"]

# функция для вставки навыков по тексту
def enrich_with_skills_inline(text, n_min=2, n_max=3):
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    k = random.randint(n_min, n_max)
    skills = random.sample(sorted(IT_SKILLS), k)
    for skill in skills:
        idx = random.randint(0, len(sentences)-1)
        sentences[idx] += f" {skill}"
    return " ".join(sentences)

# функция для вставки gender mismatch слов
def insert_gender_mismatch(text):
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    word = random.choice(GENDER_MISMATCH_WORDS)
    idx = random.randint(0, len(sentences)-1)
    sentences[idx] += f" {word}"
    return " ".join(sentences)

# функция для транслитерации
def random_translit(word):
    result = ""
    for ch in word:
        if random.random() < 0.5:
            if "а" <= ch <= "я":
                result += chr(random.randint(97, 122))
            elif "a" <= ch <= "z":
                result += chr(random.randint(1072, 1103))
            else:
                result += ch
        else:
            result += ch
    return result

def add_translit_noise(text):
    words = text.split()
    if not words:
        return text
    idx = random.randint(0, len(words)-1)
    words[idx] = random_translit(words[idx])
    return " ".join(words)

# добавляем новые письма в датасет
cl_data["letter_with_skills"] = cl_data["old_output"].astype(str)

# выбираем письма для каждой вставки
num_it = int(len(cl_data) * 0.2)      # 20% писем получат IT-навыки
num_gender = int(len(cl_data) * 0.5)  # 50% писем получат gender mismatch
num_translit = int(len(cl_data) * 0.5)  # 50% писем с транслитом

it_indices = random.sample(range(len(cl_data)), num_it)
gender_indices = random.sample(range(len(cl_data)), num_gender)
translit_indices = random.sample(range(len(cl_data)), num_translit)

# вставляем навыки
for i in it_indices:
    cl_data.at[i, "letter_with_skills"] = enrich_with_skills_inline(str(cl_data.at[i, "old_output"]))

# вставляем gender mismatch
for i in gender_indices:
    current_text = cl_data.at[i, "letter_with_skills"]
    cl_data.at[i, "letter_with_skills"] = insert_gender_mismatch(current_text)

# добавляем транслитерацию
for i in translit_indices:
    cl_data.at[i, "letter_with_skills"] = add_translit_noise(cl_data.at[i, "letter_with_skills"])

print("Навыки, gender mismatch и транслитерация добавлены в письма")
print(cl_data[["old_output", "letter_with_skills"]].head())

Навыки, gender mismatch и транслитерация добавлены в письма
                                          old_output  \
0  Добрый день!\n\nМеня зовут Эдиге, и я хочу отк...   
1  Добрый день!\n\nМеня зовут Диёр, и я решил отк...   
2  Добрый день!\n\nМеня зовут Акимали, и я хочу о...   
3  Добрый день!\n\nМеня зовут Давид, и я хочу отк...   
4  Добрый день!\n\nМеня зовут Диёр, и я решил отк...   

                                  letter_with_skills  
0  Добрый день!\n\nМеня зовут Эдиге, и я хочу отк...  
1  Добрый день! Меня зовут Диёр, и я решил отклик...  
2  Добрый день! Меня зовут Акимали, и я хочу откл...  
3  Добрый день! Меня зовут Давид, и я хочу отклик...  
4  Добрый день! Меня зовут Диёр, и я решил отклик...  


In [16]:
import re
import pandas as pd

# нормализация
def norm_e(text: str) -> str:
    return text.replace("Ё", "Е").replace("ё", "е")

REGEX_REPLACEMENTS = [
    (r"c\+\+|cpp", "cpp"),
    (r"c#", "csharp"),
    (r"\.net|dotnet", "dotnet"),
    (r"node\.?js", "nodejs"),
    (r"postgresql|postgres|psql", "postgres"),
    (r"scikit-?learn|sk-?learn", "sklearn"),
    (r"pytorch|torch", "pytorch"),
    (r"javascript|js", "js"),
    (r"typescript|ts", "typescript"),
    (r"google cloud|gcp", "gcp"),
    (r"amazon web services|aws", "aws"),
    (r"objective-c", "objectivec"),
]

SYNONYMS = {
    "питон": "python", "пайтон": "python",
    "джанго": "django", "фласк": "flask",
    "реакт": "react", "ангуляр": "angular", "вью": "vue",
    "кубернетес": "kubernetes", "кубер": "kubernetes",
    "постгрес": "postgres", "монго": "mongodb",
    "редис": "redis", "кликхаус": "clickhouse",
    "фастапи": "fastapi", "джава": "java",
    "голанг": "go", "го": "go",
    "пандас": "pandas", "нампай": "numpy",
    "кафка": "kafka",
}

def normalize_text_for_bow(text: str) -> str:
    text = norm_e(text.lower())
    for pat, repl in REGEX_REPLACEMENTS:
        text = re.sub(pat, repl, text)
    text = re.sub(r"[^a-zа-я0-9#+.\s]", " ", text)
    tokens = text.split()
    normalized = [SYNONYMS.get(t, t) for t in tokens]
    return " ".join(normalized)

def extract_words(text: str) -> set:
    return set(normalize_text_for_bow(text).split())

# аномалии языка
def detect_transliteration_anomalies(text: str):
    transliteration_map = {
        'pochemu': 'почему', 'kak': 'как', 'chto': 'что', 'gde': 'где',
        'kogda': 'когда', 'kakoy': 'какой', 'kakaya': 'какая',
        'kakoe': 'какое', 'kakie': 'какие', 'kto': 'кто',
        'chego': 'чего', 'chemu': 'чему', 'chem': 'чем',
        'kogo': 'кого', 'komu': 'кому', 'kom': 'ком',
    }
    anomalies = []
    words = re.findall(r'\b\w+\b', text.lower())
    for word in words:
        if word in transliteration_map:
            anomalies.append({
                'word': word,
                'correct_word': transliteration_map[word],
                'position': text.lower().find(word),
                'anomaly_type': 'transliteration'
            })
    return anomalies

def detect_mixed_language_anomalies(text: str):
    cyrillic_pattern = re.compile(r'[а-яё]', re.IGNORECASE)
    latin_pattern = re.compile(r'[a-z]', re.IGNORECASE)
    words = re.findall(r'\b\w+\b', text.lower())
    anomalies, total_words = [], len(words)

    for word in words:
        if len(word) < 2:
            continue
        has_cyrillic = bool(cyrillic_pattern.search(word))
        has_latin = bool(latin_pattern.search(word))
        if has_cyrillic and has_latin:
            cyrillic_chars = len(cyrillic_pattern.findall(word))
            latin_chars = len(latin_pattern.findall(word))
            if cyrillic_chars >= 2 and latin_chars >= 2:
                anomalies.append({
                    'word': word,
                    'position': text.lower().find(word),
                    'cyrillic_chars': cyrillic_chars,
                    'latin_chars': latin_chars,
                    'anomaly_type': 'mixed_script'
                })

    anomalies.extend(detect_transliteration_anomalies(text))
    anomaly_count = len(anomalies)
    anomaly_rate = (anomaly_count / total_words * 100) if total_words > 0 else 0

    return {
        'has_anomalies': anomaly_count > 0,
        'anomalies': anomalies,
        'anomaly_count': anomaly_count,
        'anomaly_rate': round(anomaly_rate, 2),
        'total_words': total_words
    }

def get_anomaly_severity(anomaly_rate: float) -> str:
    if anomaly_rate == 0:
        return 'none'
    elif anomaly_rate < 5:
        return 'low'
    elif anomaly_rate < 15:
        return 'medium'
    elif anomaly_rate < 30:
        return 'high'
    else:
        return 'critical'

MASC_PATTERNS = [r"\bготов\b", r"\bуверен\b", r"\bзаинтересован\b", r"\bмотивирован\b",
    r"\bвдохновл[ее]н\b", r"\bнацелен\b", r"\bженат\b", r"\bработал\b"]
FEM_PATTERNS = [r"\bготова\b", r"\bуверена\b", r"\bзаинтересована\b", r"\bмотивирована\b",
    r"\bвдохновл[ее]на\b", r"\bнацелена\b", r"\bзамужем\b", r"\bработала\b"]

def gender_mismatch(text: str, gender: str) -> bool:
    t = norm_e(text.lower())
    if gender.strip().lower() == "мужской":
        return any(re.search(p, t) for p in FEM_PATTERNS)
    if gender.strip().lower() == "женский":
        return any(re.search(p, t) for p in MASC_PATTERNS)
    return False

# новые навыки
def detect_new_skills(exp_text, job_text, letter_text):
    exp_tokens = {t for t in extract_words(exp_text) if t in IT_SKILLS}
    job_tokens = {t for t in extract_words(job_text) if t in IT_SKILLS}
    letter_tokens = {t for t in extract_words(letter_text) if t in IT_SKILLS}

    allowed_skills = exp_tokens & job_tokens
    matched_job = letter_tokens & allowed_skills
    new_skills = letter_tokens - exp_tokens
    new_and_relevant = new_skills & job_tokens

    return matched_job, new_skills, new_and_relevant

def gender_mismatch(text: str) -> bool:
    """
    Возвращает True, если в письме встречаются слова обоих родов.
    """
    t = norm_e(text.lower())
    has_masc = any(re.search(p, t) for p in MASC_PATTERNS)
    has_fem = any(re.search(p, t) for p in FEM_PATTERNS)
    return has_masc and has_fem



# применение
rows = []
for idx, r in cl_data.iterrows():
    job, exp, letter = r["job_description"], r["user_experience"], r["letter_with_skills"]
    gender = r.get("gender_clean", "")

    anomaly_info = detect_mixed_language_anomalies(letter)
    latin_flag = int(anomaly_info['has_anomalies'])
    anomaly_severity = get_anomaly_severity(anomaly_info['anomaly_rate'])

    gender_flag = gender_mismatch(letter)
    matched_job, new_skills, new_and_relevant = detect_new_skills(exp, job, letter)
    job_len = len(extract_words(job)) + 1
    new_skills_ratio = len(new_skills) / job_len

    anomaly_flag = int(latin_flag or gender_flag or new_skills_ratio > 0.01)

    rows.append({
        "latin_mixed_label": latin_flag,
        "gender_mismatch_label": int(gender_flag),
        "extra_skills_count": len(new_skills),
        "matched_job_skills": len(matched_job),
        "new_skills_list": ",".join(sorted(new_skills)) if new_skills else "",
        "new_and_relevant_list": ",".join(sorted(new_and_relevant)) if new_and_relevant else "",
        "new_skills_ratio": new_skills_ratio,
        "anomaly_flag": anomaly_flag,
        "anomaly_severity": anomaly_severity
    })

new_features = pd.DataFrame(rows, index=cl_data.index)
cl_data = pd.concat([cl_data, new_features], axis=1)

print("Все признаки аномалий добавлены")
print(cl_data["anomaly_flag"].value_counts())


Все признаки аномалий добавлены


ValueError: Grouper for 'anomaly_flag' not 1-dimensional

In [8]:
examples = cl_data[cl_data["anomaly_flag"] == 1].head(20)

for i, row in examples.iterrows():
    print(f"\n=== Пример {i} ===")
    print("1.Текст письма:", row["letter_with_skills"][:300], "...")
    print("2.Аномалии языка:", detect_mixed_language_anomalies(row["letter_with_skills"]))
    print("3.Gender mismatch:", row["gender_mismatch_label"])
    print("4.Новые навыки:", row["new_skills_list"])
    print("5.severity:", row["anomaly_severity"])



=== Пример 0 ===
1.Текст письма: Добрый день!

Меня зовут Эдиге, и я хочу откликнуться на вакансию Python-разработчика в подразделении искусственного интеллекта и анализа данных. Вот пять причин, почему мы подходим друг другу.

1. Мой технический опыт полностью соответствует требованиям вакансии. Я работал в Epam Kazakhstan, ByteGe ...
2.Аномалии языка: {'has_anomalies': False, 'anomalies': [], 'anomaly_count': 0, 'anomaly_rate': 0.0, 'total_words': 227}
3.Gender mismatch: 0
4.Новые навыки: kubernetes,mongodb
5.severity: none

=== Пример 3 ===
1.Текст письма: Добрый день! Меня зовут Давид, и я хочу откликнуться на вакансию тестировщика в вашей компании. Вот пять причин, почему мы подходим друг другу: 1. Я имею практический опыт тестирования в компании Яндекс, где работал над различными проектами: "Яндекс Маршруты", каршеринг, "Яндекс.Метро" и API "Яндекс ...
2.Аномалии языка: {'has_anomalies': True, 'anomalies': [{'word': 'poчдmaь', 'position': 1105, 'cyrillic_chars': 3, 'latin_chars'

In [9]:
# Сохраняем размеченный датасет
cl_data.to_csv("cl_data_annotated.csv", index=False)
print("Размеченный датасет сохранён: cl_data_annotated.csv")

Размеченный датасет сохранён: cl_data_annotated.csv


---
1 model
---

In [10]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.2 MB/s eta 0:00:00


In [11]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import pickle
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from scipy.sparse import hstack

In [12]:
# загружаем размеченный датасет
df = pd.read_csv("cl_data_annotated.csv")
df.fillna("", inplace=True)

# Числовые фичи
numeric_features = ["latin_mixed_label", "gender_mismatch_label",
                    "extra_skills_count", "matched_job_skills",
                    "new_skills_ratio"]

X_numeric = df[numeric_features]
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# TF-IDF на тексте cover letter (
tfidf = TfidfVectorizer(max_features=5000)
X_text = tfidf.fit_transform(df["old_output"])

# Сохраним TF-IDF
with open("tfidf_post_1.pkl", "wb") as f:
    pickle.dump(tfidf, f)

# Объединяем числовые и текстовые фичи
X = hstack([X_text, X_numeric_scaled])
y = df["anomaly_flag"]  # бинарная метка: 0 = нет аномалии, 1 = аномалия

# train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Подсчёт sample_weight для баланса классов
from collections import Counter
class_counts = Counter(y_train)
total = sum(class_counts.values())
scale_pos_weight = {cls: total / (len(class_counts) * count) for cls, count in class_counts.items()}
sample_weight = y_train.map(scale_pos_weight)

print("scale_pos_weight:", scale_pos_weight)

# Обучение CatBoost
model = CatBoostClassifier(
    iterations=300,
    depth=6,
    learning_rate=0.1,
    loss_function='Logloss',
    random_seed=42,
    verbose=50
)

model.fit(X_train, y_train, sample_weight=sample_weight)

# Оценка модели
y_pred = model.predict(X_test)
print("Classification report:\n")
print(classification_report(y_test, y_pred))
print("\nConfusion matrix:\n")
print(confusion_matrix(y_test, y_pred))

# Важность фичей
importances = model.feature_importances_
feature_names = list(tfidf.get_feature_names_out()) + numeric_features
indices = np.argsort(importances)[::-1]

print("\nTop 20 feature importances:")
for i in range(20):
    print(f"{feature_names[indices[i]]}: {importances[indices[i]]:.4f}")

model.save_model("cat_model_post.cbm")


/tmp/ipython-input-2270990247.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)


scale_pos_weight: {0: 1.0759829968119021, 1: 0.934040590405904}
0:	learn: 0.5047971	total: 665ms	remaining: 3m 18s
50:	learn: 0.0028124	total: 23.3s	remaining: 1m 53s
100:	learn: 0.0026332	total: 36s	remaining: 1m 10s
150:	learn: 0.0026327	total: 47.5s	remaining: 46.9s
200:	learn: 0.0026317	total: 59.1s	remaining: 29.1s
250:	learn: 0.0026307	total: 1m 10s	remaining: 13.8s
299:	learn: 0.0026301	total: 1m 21s	remaining: 0us
Classification report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       235
           1       1.00      1.00      1.00       272

    accuracy                           1.00       507
   macro avg       1.00      1.00      1.00       507
weighted avg       1.00      1.00      1.00       507


Confusion matrix:

[[235   0]
 [  1 271]]

Top 20 feature importances:
latin_mixed_label: 47.2331
new_skills_ratio: 45.9269
gender_mismatch_label: 4.6675
стека: 0.1406
extra_skills_count: 0.1381
готова: 0.1068
задач: 0.099

---
Сопроводительное письмо
---

In [14]:
import pandas as pd
import pickle
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from catboost import CatBoostClassifier

annotated = pd.read_csv("cl_data_annotated.csv")

# Train/test split
train_df, test_df = train_test_split(annotated, test_size=0.2, random_state=42)

# Model 1 и TF-IDF
model1 = CatBoostClassifier()
model1.load_model("cat_model_post.cbm")

with open("tfidf_post_1.pkl", "rb") as f:
    tfidf1 = pickle.load(f)

# Входы для Model 1
# TF-IDF по письмам (с фичами)
X1_train_text = tfidf1.transform(train_df["letter_with_skills"].astype(str))
X1_test_text  = tfidf1.transform(test_df["letter_with_skills"].astype(str))

numeric_features = [
    "latin_mixed_label",
    "gender_mismatch_label",
    "extra_skills_count",
    "matched_job_skills",
    "new_skills_ratio"
]

X1_train_num = train_df[numeric_features].values
X1_test_num  = test_df[numeric_features].values

X1_train = sp.hstack([X1_train_text, X1_train_num])
X1_test  = sp.hstack([X1_test_text,  X1_test_num])

# Получаем вероятности от Model 1
train_proba = model1.predict_proba(X1_train)
test_proba  = model1.predict_proba(X1_test)

# Текст для Model 2 = только сопроводительное письмо
texts2_train = train_df["letter_with_skills"].astype(str)
texts2_test  = test_df["letter_with_skills"].astype(str)

tfidf2 = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X2_train_text = tfidf2.fit_transform(texts2_train)
X2_test_text  = tfidf2.transform(texts2_test)

# финальные входы для Model 2
X2_train = sp.hstack([X2_train_text, train_proba])
X2_test  = sp.hstack([X2_test_text,  test_proba])

y2_train = train_df["anomaly_flag"]
y2_test  = test_df["anomaly_flag"]

# Баланс классов
class_counts = y2_train.value_counts().to_dict()
N = len(y2_train)
K = y2_train.nunique()
class_weights = [N / (K * class_counts.get(c, 1)) for c in sorted(class_counts.keys())]

print("Рассчитанные class_weights:", class_weights)

# Обучение Model 2
model2 = CatBoostClassifier(
    iterations=300,
    depth=6,
    learning_rate=0.1,
    loss_function='MultiClass',
    class_weights=class_weights,
    random_seed=42,
    verbose=50
)
model2.fit(X2_train, y2_train)

# Оценка
y2_pred = model2.predict(X2_test)
print("Model 2 report:\n", classification_report(y2_test, y2_pred))
print("Confusion matrix:\n", confusion_matrix(y2_test, y2_pred))

# Сохранение Model 2
model2.save_model("cat_model_pre.cbm")
with open("tfidf_pre.pkl", "wb") as f:
    pickle.dump(tfidf2, f)


Рассчитанные class_weights: [1.0828877005347595, 0.9288990825688074]
0:	learn: 0.6580797	total: 881ms	remaining: 4m 23s
50:	learn: 0.4005625	total: 35.6s	remaining: 2m 53s
100:	learn: 0.3416145	total: 1m 7s	remaining: 2m 13s
150:	learn: 0.2873773	total: 1m 40s	remaining: 1m 39s
200:	learn: 0.2488662	total: 2m 12s	remaining: 1m 5s
250:	learn: 0.2245490	total: 2m 45s	remaining: 32.4s
299:	learn: 0.2031629	total: 3m 16s	remaining: 0us
Model 2 report:
               precision    recall  f1-score   support

           0       0.72      0.93      0.81       241
           1       0.92      0.67      0.77       266

    accuracy                           0.79       507
   macro avg       0.82      0.80      0.79       507
weighted avg       0.82      0.79      0.79       507

Confusion matrix:
 [[225  16]
 [ 88 178]]
